In [3]:
import pandas as pd
purchase_behaviour = pd.read_csv('purchase_behaviour.csv')
transaction_data = pd.read_csv('transaction_data.csv')
print(purchase_behaviour.isnull().sum())
print(transaction_data.isnull().sum())

LYLTY_CARD_NBR      0
LIFESTAGE           0
PREMIUM_CUSTOMER    0
dtype: int64
DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64


In [4]:
transaction_data['DATE'] = pd.to_datetime(transaction_data['DATE'])
transaction_data['TOT_SALES'] = pd.to_numeric(transaction_data['TOT_SALES'])
merged_data = pd.merge(transaction_data, purchase_behaviour, on='LYLTY_CARD_NBR', how='inner')

In [5]:
product_profitability = merged_data.groupby('PROD_NAME')['TOT_SALES'].sum().reset_index()
top_3_products = product_profitability.sort_values(by='TOT_SALES', ascending=False).head(3)
print("Top 3 Most Profitable Products:")
print(top_3_products)

Top 3 Most Profitable Products:
                                   PROD_NAME  TOT_SALES
11          Dorito Corn Chp     Supreme 380g    40352.0
86    Smiths Crnkle Chip  Orgnl Big Bag 380g    36367.6
77  Smiths Crinkle Chips Salt & Vinegar 330g    34804.2


In [6]:
customer_metrics = merged_data.groupby('LYLTY_CARD_NBR').agg(
    total_spend=('TOT_SALES', 'sum'),
    purchase_frequency=('TXN_ID', 'nunique'),
    last_purchase_date=('DATE', 'max')
).reset_index()

customer_metrics['loyalty_score'] = customer_metrics['total_spend'] * customer_metrics['purchase_frequency']
loyal_customers = customer_metrics.sort_values(by='loyalty_score', ascending=False).head(int(len(customer_metrics) * 0.1))
loyal_customer_demographics = pd.merge(loyal_customers, purchase_behaviour, on='LYLTY_CARD_NBR', how='left')

loyal_customer_summary = loyal_customer_demographics.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER']).size().reset_index(name='count')
print("Characteristics of Loyal Customers:")
print(loyal_customer_summary)

Characteristics of Loyal Customers:
                 LIFESTAGE PREMIUM_CUSTOMER  count
0   MIDAGE SINGLES/COUPLES           Budget     80
1   MIDAGE SINGLES/COUPLES       Mainstream    246
2   MIDAGE SINGLES/COUPLES          Premium    150
3             NEW FAMILIES           Budget      2
4           OLDER FAMILIES           Budget   1262
5           OLDER FAMILIES       Mainstream    793
6           OLDER FAMILIES          Premium    598
7    OLDER SINGLES/COUPLES           Budget    316
8    OLDER SINGLES/COUPLES       Mainstream    299
9    OLDER SINGLES/COUPLES          Premium    295
10                RETIREES           Budget    169
11                RETIREES       Mainstream    213
12                RETIREES          Premium    134
13          YOUNG FAMILIES           Budget   1015
14          YOUNG FAMILIES       Mainstream    634
15          YOUNG FAMILIES          Premium    604
16   YOUNG SINGLES/COUPLES           Budget    105
17   YOUNG SINGLES/COUPLES       Mainstream   